# Marian notebook


https://towardsdatascience.com/normalization-vs-standardization-quantitative-analysis-a91e8a79cebf
https://www.kaggle.com/code/junkal/selecting-the-best-regression-model



Load la data comme dans baseline


In [2]:
import csv
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso





from sklearn.dummy import DummyRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error
from verstack.stratified_continuous_split import scsplit # pip install verstack
import nltk# cant't find the stopword corpus otherwise
nltk.download('stopwords')
from nltk.corpus import stopwords 

import matplotlib.pyplot as plt



# Load the training data
train_data = pd.read_csv("data/train.csv")

# Load the evaluation data
eval_data = pd.read_csv("data/evaluation.csv")


# Here we split our training data into trainig and testing set. This way we can estimate the evaluation of our model without uploading to Kaggle and avoid overfitting over our evaluation dataset.
# scsplit method is used in order to split our regression data in a stratisfied way and keep a similar distribution of retweet counts between the two sets
X_train, X_test, y_train, y_test = scsplit(train_data, train_data['retweets_count'], stratify=train_data['retweets_count'], train_size=0.7, test_size=0.3)
# We remove the actual number of retweets from our features since it is the value that we are trying to predict
X_train = X_train.drop(['retweets_count'], axis=1)
X_test = X_test.drop(['retweets_count'], axis=1)

# You can examine the available features using X_train.head()
X_train.head()#le premier numero correspond au no de la ligne dans le csv



c:\Users\maria\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maria\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,favorites_count,followers_count,statuses_count,friends_count,mentions,urls,verified,hashtags,timestamp,TweetID
37857,rt propos choquant part candidat stagiaires,0,10010,892820,5647,[],[],0,"['zemmour', 'presidentielles2022', 'entreprises']",1646971823000,927516
169760,rt macron mec vendu armes pointe poutine viola...,0,17883,50369,4053,[],[],0,[],1647285309000,1121506
313976,retweete mourir faim peuple gilbert perrincite...,0,1735,29662,5001,[],['https://t.co/flmd5dqwfl'],0,[],1646900368000,916095
79597,rt gauche libérale terminé,0,4296,790752,1,[],[],0,['melenchon2dtour'],1647456722000,75212
105127,choisir pen mélenchon choisirait pen bourgeois...,0,400,19170,276,[],['https://t.co/jfr8wtuil8'],0,"['presidentielle2022', 'enthoven', 'melenchon']",1647631463000,1131272


# Data cleaning


Feature selection
- mentions is always empty -> DROP Mention
- url -> can't be used as plain text -> should count the number of URLs in URLs_count
- hashtags -> first: count the number of hashtags in hashtags_count-> hashtag count is not relevant: small correlation with target + make RF score increase
            -> second: 
- remove tweetID -> TO DO

# Data processing


To do:
- data normalization
- feature extraction and selection

Contenu texte du tweet: NAN?????

Autres features vectorisées

In [3]:
#mentions is always empty -> DROP Mention

X_train = X_train.drop(['mentions'], axis=1)
X_test = X_test.drop(['mentions'], axis=1)

#Tweet id is not relevant -> DROP Tweet id
X_train = X_train.drop(['TweetID'], axis=1)
X_test = X_test.drop(['TweetID'], axis=1)


#add a column to data which counts url
X_train['url_count'] = X_train['urls'].str.count('http')
X_test['url_count'] = X_test['urls'].str.count('http')
X_train = X_train.drop(['urls'], axis=1)
X_test = X_test.drop(['urls'], axis=1)

#drop the text column
X_train = X_train.drop(['text'], axis=1)
X_test = X_test.drop(['text'], axis=1)
#frop hashtags
X_train = X_train.drop(['hashtags'], axis=1)
X_test = X_test.drop(['hashtags'], axis=1)


#add followers/friends
X_train['followers_friends'] = X_train['followers_count'] / \
        (X_train['friends_count'].apply(lambda x: x+1))
X_test['followers_friends'] = X_test['followers_count'] / \
        (X_test['friends_count'].apply(lambda x: x+1))  






# Data visualization and selection

Source: https://amueller.github.io/aml/05-advanced-topics/12-feature-selection.html


#### Supervised 2
Mutual information

Source: 
https://machinelearningmastery.com/feature-selection-for-regression-data/


# example of mutual information feature selection for numerical input data
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from matplotlib import pyplot




# configure to select all features
fs = SelectKBest(score_func=mutual_info_regression, k='all')
# learn relationship from training data
fs.fit(X_train, y_train)
for i in range(len(fs.scores_)):
	#print name feature i, score
	print( X_train.columns[i], fs.scores_[i])




## RANDOM FOREEEST

source for feature selection: https://amueller.github.io/aml/05-advanced-topics/12-feature-selection.html

Feature selection  for random forest


reg2 = RandomForestRegressor() 


from sklearn.feature_selection import RFE
for i in range(1, len(X_train.columns)+1):
    select = RFE(reg2,n_features_to_select=i)
    select.fit(X_train, y_train)
    #print selected features
    print(X_train.columns[select.support_])

    #print mean absolute error
    y_pred=select.predict(X_test)
    y_pred = [int(value) if value >= 0 else 0 for value in y_pred]
    print(i,'features: Prediction error Random Forest:',mean_absolute_error(y_test, y_pred))



#20 min per features number



Index(['favorites_count'], dtype='object')
1 features: Prediction error Random Forest: 8.134917271708526

Index(['favorites_count', 'timestamp'], dtype='object')
2 features: Prediction error Random Forest: 6.82604928854611

Index(['favorites_count', 'followers_count', 'timestamp'], dtype='object')
3 features: Prediction error Random Forest: 6.569370285617425

Index(['favorites_count', 'followers_count', 'statuses_count', 'timestamp'], dtype='object')
4 features: Prediction error Random Forest: 6.490728969498357

Index(['favorites_count', 'followers_count', 'statuses_count', 'friends_count',
       'timestamp'],
      dtype='object')
5 features: Prediction error Random Forest: 6.362526014445669

Index(['favorites_count', 'followers_count', 'statuses_count', 'friends_count',
       'timestamp', 'url_count'],
      dtype='object')
6 features: Prediction error Random Forest: 6.212918232241904
Index(['favorites_count', 'followers_count', 'statuses_count', 'friends_count',
       'verified', 'timestamp', 'url_count'],
      dtype='object')
7 features: Prediction error Random Forest: 6.1723498224896645

Index(['favorites_count', 'followers_count', 'statuses_count', 'friends_count',
       'verified', 'timestamp', 'url_count', 'hashtag_count'],
      dtype='object')
8 features: Prediction error Random Forest: 6.234285391417352

In [4]:
X_train.head()

,favorites_count,followers_count,statuses_count,friends_count,verified,timestamp,url_count,followers_friends
37857,0,10010,892820,5647,0,1646971823000,0,1.772309
169760,0,17883,50369,4053,0,1647285309000,0,4.411199
313976,0,1735,29662,5001,0,1646900368000,1,0.346861
79597,0,4296,790752,1,0,1647456722000,0,2148.000000
105127,0,400,19170,276,0,1647631463000,1,1.444043


In [5]:

col= ['followers_count', 'friends_count','statuses_count', 'favorites_count','verified', 'followers/friends', 'url_count','timestamp']
								


remove=['verified']




In [1]:
from flaml import AutoML
automl = AutoML()

settings = {
    "time_budget": 300,  # total running time in seconds
    "metric": 'mae',  # can be: 'r2', 'rmse', 'mae', 'mse', 'accuracy', 'roc_auc', 'roc_auc_ovr',
    # 'roc_auc_ovo', 'log_loss', 'mape', 'f1', 'ap', 'ndcg', 'micro_f1', 'macro_f1'
    "task": 'regression',  # task type
    # ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']
    'estimator_list': ['rf'],
    "seed": 7654321,    # random seed
}

automl.fit(X_train.drop(remove, axis=1), y_train, **settings)
# Print the best model
print(automl.model.estimator)

c:\Users\maria\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


NameError: name 'X_train' is not defined

In [7]:
from sklearn.ensemble import RandomForestRegressor
reg1 = RandomForestRegressor(max_features=0.8112248359805823, max_leaf_nodes=7086,
                      n_estimators=33, n_jobs=-1,criterion='absolute_error')
# We fit our model using the training data
reg1.fit(X_train.drop(remove,axis=1), y_train)
# And then we predict the values for our testing set
y_pred = reg1.predict(X_test.drop(remove,axis=1))
# We want to make sure that all predictions are non-negative integers
y_pred = [round(value) if value >= 0 else 0 for value in y_pred]

print("Prediction error :", mean_absolute_error(y_true=y_test, y_pred=y_pred))